In [12]:
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn import metrics

import json
import os
import pandas as pd

from util import gen_model_preds_df

groups_s = json.load(open("SWDA_dialogue-acts-groups.json"))
groups_a = json.load(open("AMI-DA_dialogue-acts-groups.json"))

def report_metrics(frames, conditions, group=False):
    metric_funcs = [
        lambda x,y: metrics.precision_score(x,y,average='macro'), 
        lambda x,y: metrics.recall_score(x,y,average='macro'), 
        lambda x,y: metrics.f1_score(x,y,average='macro'),
        lambda x,y: metrics.precision_score(x,y,average='micro')]
    metric_names = [
        'macro precision',
        'macro recall',
        'macro f1',
        'micro accuracy']
    if not group:
        table = [[
            metric(df['da_tag'], df[cond])
                for df in frames]
                for cond in conditions for metric in metric_funcs]
    else:
        table = [[
            metric([groups[ix][dat] for dat in df['da_tag']], 
                   [groups[ix][dat] for dat in df[cond]])
                for ix,df in enumerate(frames)]
                for cond in conditions for metric in metric_funcs for g in groups] 
    
    multiindex = [[c for c in conditions for m in metric_names],
        [m for c in conditions for m in metric_names]]
    return pd.DataFrame(table, columns=['SWBD', 'AMI'], index=multiindex)

conditions = ['NL_bert', 'L_bert', 'NL_cnn', 'L_cnn']

model_dirs = [f'/scratch/DistributionalDiscourse/models/SWDA-{c}_2019-11-20/' for c in conditions]
dfs = gen_model_preds_df('SWDA', conditions, model_dirs, group=False)
display(dfs.head(10))

model_dirs = [f'/scratch/DistributionalDiscourse/models/AMI-DA-{c}_2019-11-20/' for c in conditions]
dfa = gen_model_preds_df('AMI-DA', conditions, model_dirs, group=False)
display(dfa.head(10))
dfa = dfa[dfa['da_tag'].notnull()]

speaker                                                utt  \
dialogue_id utt_no                                                              
sw2608      0            A                                             Okay .   
            1            B                          Do you want to go first ?   
            2            A                                 You can go first ,   
            3            A                                        or I will .   
            4            B                              Well , you go ahead .   
            5            A                                             Okay .   
            6            A  Well , I'm going to tell you what I'd have <la...   
            7            B                                       <laughter> .   
            8            A      Down in the south , we have a lot of shrimp ,   
            9            B                                           # Oh , #   

                             da_tag          NL_bert           L_bert  \
dialogue_id utt_no                                                      
sw2608      0       fo_o_fw_"_by_bc  fo_o_fw_"_by_bc  fo_o_fw_"_by_bc   
            1                    qy               qy               qy   
            2              oo_co_cc               ad               ad   
            3              oo_co_cc               sd               sd   
            4                    ad               ad               ad   
            5                    aa                b                b   
            6                    sd               ^h               sd   
            7                     x                x                x   
            8                    sd               sd               sd   
            9                     b                b                b   

                             NL_cnn            L_cnn  
dialogue_id utt_no                                    
sw2608      0       fo_o_fw_"_by_bc  fo_o_fw_"_by_bc  
            1                    qy               qy  
            2                    sd               sd  
            3                    sd               sd  
            4                    ad               sd  
            5                    bk               bk  
            6                    sd               sd  
            7                     x                x  
            8                    sd               sd  
            9                     b                b

speaker                                         utt  \
dialogue_id utt_no                                                       
IS1001d     0            A                                      Okay .   
            1            A           Je croix que c'est dommage de le    
            2            A  it will be sad to destroy this prototype .   
            3            A             It really looks like a banana .   
            4            C                            It is a banana .   
            5            A                 It is a banana . <laughter>   
            6            C              It is the essence of bananas .   
            7            C       I would be confused with this thing .   
            8            A                                               
            9            C                                               

                       da_tag    NL_bert     L_bert    NL_cnn     L_cnn  
dialogue_id utt_no                                                       
IS1001d     0       ami_da_16   ami_da_2   ami_da_9  ami_da_9  ami_da_9  
            1        ami_da_3  ami_da_14  ami_da_16  ami_da_4  ami_da_4  
            2       ami_da_16  ami_da_14  ami_da_14  ami_da_4  ami_da_4  
            3        ami_da_4   ami_da_9   ami_da_9  ami_da_9  ami_da_9  
            4       ami_da_14   ami_da_9   ami_da_4  ami_da_4  ami_da_9  
            5       ami_da_12   ami_da_9   ami_da_4  ami_da_4  ami_da_9  
            6       ami_da_14   ami_da_9   ami_da_9  ami_da_4  ami_da_4  
            7       ami_da_15   ami_da_4   ami_da_4  ami_da_9  ami_da_4  
            8        ami_da_3   ami_da_3   ami_da_3  ami_da_3  ami_da_3  
            9        ami_da_3   ami_da_3   ami_da_3  ami_da_3  ami_da_3

In [8]:
metric_funcs = [
#     lambda x,y: metrics.precision_score(x,y,average='macro'), 
#     lambda x,y: metrics.recall_score(x,y,average='macro'), 
#     lambda x,y: metrics.f1_score(x,y,average='micro'),
    lambda x,y: metrics.precision_score(x,y,average='micro')]
for cond in conditions:
    print(cond)
    for g in ['Forward-Communicative-Function', 'Backwards-Communicative-Function']:
        print(g)
        dfsf = dfs[dfs['da_tag'] == g]
        for mf in metric_funcs:
            print(f"{mf(dfsf['da_tag'],dfsf[cond])*100:.2f}")
    print()

NL_bert
Forward-Communicative-Function
92.54
Backwards-Communicative-Function
91.67

L_bert
Forward-Communicative-Function
95.42
Backwards-Communicative-Function
90.91

NL_cnn
Forward-Communicative-Function
93.30
Backwards-Communicative-Function
90.85

L_cnn
Forward-Communicative-Function
94.64
Backwards-Communicative-Function
90.13



In [9]:
for cond in conditions:
    print(cond)
    for g in ['Forward-Communicative-Function', 'Backwards-Communicative-Function']:
        print(g)
        dfaf = dfa[dfa['da_tag'] == g]
        for mf in metric_funcs:
            print(f"{mf(dfaf['da_tag'],dfaf[cond])*100:.2f}")
    print()

NL_bert
Forward-Communicative-Function
87.77
Backwards-Communicative-Function
80.83

L_bert
Forward-Communicative-Function
87.36
Backwards-Communicative-Function
79.87

NL_cnn
Forward-Communicative-Function
91.85
Backwards-Communicative-Function
71.21

L_cnn
Forward-Communicative-Function
91.25
Backwards-Communicative-Function
71.28



In [13]:
report_metrics([dfs,dfa], conditions)

/nix/store/2wi8nzfxniygifymcjdyvx9kskm5k2nl-python3-3.7.4-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/nix/store/2wi8nzfxniygifymcjdyvx9kskm5k2nl-python3-3.7.4-env/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


SWBD       AMI
NL_bert macro precision  0.455734  0.593820
        macro recall     0.367783  0.465963
        macro f1         0.381006  0.490873
        micro accuracy   0.770665  0.670764
L_bert  macro precision  0.561119  0.588414
        macro recall     0.430470  0.484450
        macro f1         0.459891  0.501652
        micro accuracy   0.769267  0.671383
NL_cnn  macro precision  0.556029  0.488473
        macro recall     0.343041  0.361419
        macro f1         0.372366  0.383694
        micro accuracy   0.750800  0.634698
L_cnn   macro precision  0.488935  0.500776
        macro recall     0.346400  0.371487
        macro f1         0.375904  0.379392
        micro accuracy   0.754047  0.643164